In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('../data/IRIS.csv')

In [6]:
dataset = np.array(df)

In [7]:
print(dataset[0])

[5.1 3.5 1.4 0.2 'Iris-setosa']


In [8]:
Iris_map = {'Iris-setosa':0.0, 'Iris-versicolor':1.0, 'Iris-virginica':2.0}

In [9]:
for i in range(len(dataset)):
    dataset[i][4] = Iris_map[dataset[i][4]]

In [10]:
np.random.shuffle(dataset)

In [11]:
train_dataset = dataset[:120]
test_dataset = dataset[120:]

In [12]:
x_train = tf.convert_to_tensor(train_dataset[:, :4].astype(np.float32))
y_train = tf.convert_to_tensor(train_dataset[:, 4:].astype(np.float32))
x_test = tf.convert_to_tensor(test_dataset[:, :4].astype(np.float32))
y_test = tf.convert_to_tensor(test_dataset[:,4:].astype(np.float32))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [14]:
model = Sequential()
model.add(Dense(512, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(3,activation=tf.nn.softmax))

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=40)

Epoch 1/40
4/4 [==============================] - 1s 4ms/step - loss: 1.2017 - accuracy: 0.3500
Epoch 2/40
4/4 [==============================] - 0s 4ms/step - loss: 0.9055 - accuracy: 0.5854
Epoch 3/40
4/4 [==============================] - 0s 4ms/step - loss: 0.7676 - accuracy: 0.6529
Epoch 4/40
4/4 [==============================] - 0s 4ms/step - loss: 0.6202 - accuracy: 0.7317
Epoch 5/40
4/4 [==============================] - 0s 3ms/step - loss: 0.5395 - accuracy: 0.7998
Epoch 6/40
4/4 [==============================] - 0s 4ms/step - loss: 0.4448 - accuracy: 0.8465

In [16]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 198ms/step - loss: 0.1261 - accuracy: 0.9667


[0.12607043981552124, 0.9666666388511658]

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sidda\AppData\Local\Temp\tmpzkwg_fjd\assets


In [18]:
with open('Siddarth.tflite', 'wb') as f:
  f.write(tflite_model)

In [19]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model

In [20]:
q_aware_model = quantize_model(model)

In [21]:
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=40)

Epoch 1/40
4/4 [==============================] - 0s 3ms/step - loss: 0.0656 - accuracy: 0.9667
Epoch 2/40
4/4 [==============================] - 0s 4ms/step - loss: 0.0965 - accuracy: 0.9667
Epoch 3/40
4/4 [==============================] - 0s 4ms/step - loss: 0.0962 - accuracy: 0.9667
Epoch 4/40
4/4 [==============================] - 0s 4ms/step - loss: 0.0759 - accuracy: 0.9750
Epoch 5/40
4/4 [==============================] - 0s 3ms/step - loss: 0.0936 - accuracy: 0.9750
Epoch 6/40
4/4 [==============================] - 0s 4ms/step - loss: 0.0577 - accuracy: 0.9833

In [22]:
_, baseline_accuracy = model.evaluate(x_test, y_test)
_, quantized_accuracy = q_aware_model.evaluate(x_test, y_test)

1/1 [==============================] - 1s 632ms/step - loss: 1.2338 - accuracy: 0.2667


In [23]:
print("Baseline Model Accuracy:", baseline_accuracy)
print("quantized Model Accuracy:", quantized_accuracy)

Baseline Model Accuracy: 0.9333333373069763
quantized Model Accuracy: 0.2666666805744171


In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sidda\AppData\Local\Temp\tmplff6hkwv\assets


INFO:tensorflow:Assets written to: C:\Users\sidda\AppData\Local\Temp\tmplff6hkwv\assets


In [27]:
#float fallback quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sidda\AppData\Local\Temp\tmp59dnyvex\assets


INFO:tensorflow:Assets written to: C:\Users\sidda\AppData\Local\Temp\tmp59dnyvex\assets


In [28]:
with open('Siddarth_Quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)